## Extracción de Ndds Procesadas entre dos períodos

Suponiendo un trabajo mensual de la Comisión (i.e. no existe una referencia del método ni de la frecuencia, ni de las operaciones realizadas por la Comisión, a excepción de un manual que tipifica las desagregaciones de Robo), el propósito de este Notebook es extraer las Ndds operadas por la Comisión entre dos períodos. Sea $M_1$ y $M_2$ las bases de datos donde se almacenan la información de dos perítos $t_1$ y $t_2$ (i.e. fechas de corte), entonces la $Ndd_2 - Ndd_1$, donde $Ndd_1 \in M_1 $ y $Ndd_2 \in M_2$ son el conjunto de Ndds de las respectivas bases, la diferencia de conjuntos debe dar las Ndds trabajadas entre ambas fechas de corte toda vez que $t_2 > t_1$: $Ndd_i \in M_2 \&  Ndd_i \notin M_1 $

* Debe observarse que las bases de la Comisión **siempre** contienen la información de manera acumulativa. Es decir $M_1\subset  M_2$ para $t_2 > t_1$
* Debe observarse que las bases de la Comisión (Robos)y su **metodología** para la generacion de información no dispone de una documentación de respaldo


* Coder: LeninGF
* Fecha: 2022-12-19


In [2]:
import pandas as pd
import numpy  as np
from utils import conectar_sql
import os

conx = conectar_sql()

conectando con big data database....


In [ ]:
query1 = "SELECT * FROM reportes.robos_2014_08062022;"
robos_junio_2022 = pd.read_sql(query1, conx)
robos_junio_2022.shape

In [ ]:
query2 = "SELECT * FROM reportes.robos_2014_08052022;"
robos_mayo_2022 = pd.read_sql(query2, conx)
robos_mayo_2022.shape

In [ ]:
new_ndds = set(robos_junio_2022.NDD.to_list()).difference(robos_mayo_2022.NDD.to_list())
len(new_ndds)

In [ ]:
robos_df = robos_junio_2022[robos_junio_2022.NDD.isin(new_ndds)]
robos_df.shape

In [ ]:
robos_df.head()

In [ ]:
from utils import extraer_relato
relatos_df = extraer_relato(new_ndds, conx)
relatos_df.shape

## Formateo de Relato de los hechos
Se aplican las mismas operaciones realizadas en el dataset original. Sin embargo, se debe notar que aun falta trabajar en mejorar el tokenizador al lenguaje jurídico

In [ ]:
relatos_df.RELATO = relatos_df.RELATO.str.strip()
relatos_df.RELATO = relatos_df.RELATO.str.lower()
relatos_df.RELATO = relatos_df.RELATO.str.replace("[^A-Za-z0-9áéíóúñ]+", " ", regex=True) # elimino lo que sea simbolos y me quedo con texto y numeros
relatos_df['cantidad_palabras'] = relatos_df.RELATO.str.findall(r'([\wáéíóúñ]+)').str.len()

In [ ]:
relatos_df.cantidad_palabras.hist(bins=150)

In [ ]:
import plotly_express as px
px.box(relatos_df.cantidad_palabras)

# Restriccion por el Tamanio
Para el entrenamiento del modelo se consideró que

$len(data)\in 35 < wordCound \leq 300$

In [ ]:
relatos_100df = relatos_df[(relatos_df.cantidad_palabras<=300)&(relatos_df.cantidad_palabras>35)]
relatos_100df.shape, relatos_df.shape

In [ ]:
robos_cmpl = pd.merge(robos_df, relatos_100df, on="NDD", how="inner")
robos_cmpl.shape

In [ ]:
robos_out = robos_cmpl[['NDD', 'Tipo_Delito_PJ', 'delitos_seguimiento', 'delitos_validados', 'desagregacion', 'desagregacion_1', 'RELATO']]
robos_out.shape

In [ ]:
robos_out.sample(5)

## Restricción de Tipo Penal
Se limita la información a tipo penal de Robo.
Se desconoce si  están validando tipos penales distintos de Robo
¿Por qué razón, la Comisión validaría tipos penales que no son robo, especialmente Hurto?

In [ ]:
robos_out.Tipo_Delito_PJ.value_counts()

In [ ]:
robos_out = robos_out[robos_out.Tipo_Delito_PJ=="ROBO"]
robos_out.shape

In [ ]:
robos_out.info()

In [ ]:
robos_out.delitos_seguimiento.value_counts().plot.barh()

Proponer una funcion para realizar siempre el ajuste del contenido de la variable objetivo

Eliminación de tildes

In [ ]:
robos_out.delitos_seguimiento = robos_out.delitos_seguimiento.str.replace('ECONÓMICAS', 'ECONOMICAS')
robos_out.delitos_seguimiento = robos_out.delitos_seguimiento.str.replace('VEHÍCULOS', 'VEHICULOS')

In [ ]:
robos_out.delitos_seguimiento.value_counts()

In [24]:
robos_out.to_csv(os.path.join(os.getcwd(), 'data/processed/validacionJunio2022.csv'), index=False)

: 